# 0. Configuration

In [1]:
# links to shared data MovieLens
# source on kaggle: https://www.kaggle.com/code/quangnhatbui/movie-recommender/data
RATINGS_SMALL_URL = 'https://drive.google.com/file/d/1BlZfCLLs5A13tbNSJZ1GPkHLWQOnPlE4/view?usp=share_link'
MOVIES_METADATA_URL = 'https://drive.google.com/file/d/19g6-apYbZb5D-wRj4L7aYKhxS-fDM4Fb/view?usp=share_link'

# 1. Modules and functions

In [2]:
# just to make it available to download w/o SSL verification
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

import numpy as np
import pandas as pd
import scipy.sparse as sp

from itertools import islice, cycle, product

from tqdm import tqdm_notebook
import warnings
warnings.filterwarnings('ignore')


In [3]:
!pip install implicit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## 1. 1. Helper functions to avoid copy paste

In [4]:
def read_csv_from_gdrive(url):
    """
    gets csv data from a given url (taken from file -> share -> copy link)
    :url: example https://drive.google.com/file/d/1BlZfCLLs5A13tbNSJZ1GPkHLWQOnPlE4/view?usp=share_link
    """
    file_id = url.split('/')[-2]
    file_path = 'https://drive.google.com/uc?export=download&id=' + file_id
    data = pd.read_csv(file_path)

    return data

# 2. Main

## 2.1. Load Data

`interactions` dataset shows list of movies that users watched, along with given ratings:

In [5]:
# interactions data
interactions = read_csv_from_gdrive(RATINGS_SMALL_URL)
interactions.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


`movies_metadata` dataset shows the list of movies existing on OKKO platform:

In [6]:
# information about films etc
movies_metadata = read_csv_from_gdrive(MOVIES_METADATA_URL)
movies_metadata.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


In [7]:
movies_metadata['id'] = movies_metadata['id'].astype(str)
interactions['movieId'] = interactions['movieId'].astype(str)

In [8]:
# leave only those films that intersect with each other
interactions_filtered = interactions.loc[interactions['movieId'].isin(movies_metadata['id'])]
print(interactions.shape, interactions_filtered.shape)

(100004, 4) (44989, 4)


In [9]:
interactions_filtered.ndim

2

## 2.2 Data preparation using LightFM Dataset

To use implicit kNN method `fit` we need a sparse matrix in COOrdinate format. To achieve that we will use `scipy.sparse.coo_matrix` from scipy;


In [10]:
def get_coo_matrix(
        df: pd.DataFrame, 
        user_col: str,
        item_col: str, 
        users_mapping: dict, 
        movies_mapping: dict,
        weight_col: str = None
        ):
    if weight_col is None:
        weights = np.ones(len(df), dtype=np.float32)
    else:
        weights = df[weight_col].astype(np.float32)
    interaction_matrix = sp.coo_matrix((
        weights, 
        (
            df[user_col].map(users_mapping.get), 
            df[item_col].map(movies_mapping.get)
        )
    ))
    return interaction_matrix

In [11]:
# define users mapping
users_inv_mapping = dict(enumerate(interactions_filtered['userId'].unique()))
users_mapping = {v: k for k, v in users_inv_mapping.items()}
len(users_mapping)


671

In [12]:
# define movies mapping
movies_inv_mapping = dict(enumerate(interactions_filtered['movieId'].unique()))
movies_mapping = {v: k for k, v in movies_inv_mapping.items()}
len(movies_mapping)


2830

In [13]:
# defining train set on the whole interactions dataset (as HW you will have to split into test and train for evaluation)
train_mat = get_coo_matrix(
    interactions_filtered,
    user_col = 'userId',
    item_col = 'movieId',
    users_mapping = users_mapping,
    movies_mapping = movies_mapping
    ).tocsr()

In [14]:
train_mat

<671x2830 sparse matrix of type '<class 'numpy.float32'>'
	with 44989 stored elements in Compressed Sparse Row format>

## 2.3. Model Training & Evaluation

In [`implicit`](https://pypi.org/project/implicit/), there are various models and can be groupped into:
- Item-to-Item: KNN based on various similarities - CosineRecommender, BM25Recommender, TFIDFRecommender
- implicit ALS;
- Logistic Matrix Factorization;
- Bayesian Personalized Ranking (BPR)


### 2.3.1. Train Model

In [15]:
from implicit.nearest_neighbours import (
    CosineRecommender,
    BM25Recommender,
    TFIDFRecommender
    )

Note that in item-to-item models we need to provide matrix in the form of item-user by transposing initial COO matrix user-item


In [16]:
# fit the model
cosine_model = CosineRecommender(K = 20)
cosine_model.fit(train_mat.T)


  0%|          | 0/671 [00:00<?, ?it/s]

### 2.3.2. Evaluate the Model

In [17]:
# let's make sense-check
top_N = 10
user_id = interactions_filtered['userId'].iloc[0]
row_id = users_mapping[user_id]
print(f'Rekko for user {user_id}, row number in matrix - {row_id}')

Rekko for user 1, row number in matrix - 0


In [18]:
# create mapper for movieId and title names
movie_name_mapper = dict(zip(movies_metadata['id'], movies_metadata['original_title']))

In [19]:
recs = cosine_model.recommend(
    row_id,
    train_mat,
    N = top_N,
    filter_already_liked_items = True
    )
recs = pd.DataFrame(recs).T.rename(columns = {0: 'col_id', 1: 'similarity'})
recs['inv_movie_id'] = recs['col_id'].astype(int)
recs['movieId'] = recs['inv_movie_id'].map(movies_inv_mapping.get)
recs['title'] = recs['movieId'].map(movie_name_mapper)
recs


,col_id,similarity,inv_movie_id,movieId,title
0,653.0,0.861587,653,74458,Mere Brother Ki Dulhan
1,129.0,0.844531,129,1994,The Most Dangerous Game
2,606.0,0.654064,606,8011,Highlander III: The Sorcerer
3,294.0,0.625141,294,70,Million Dollar Baby
4,337.0,0.593856,337,170,28 Days Later
5,648.0,0.577499,648,68954,Longitude
6,579.0,0.571681,579,5956,Joshua
7,399.0,0.561442,399,1088,Whale Rider
8,278.0,0.561442,278,1584,School of Rock
9,150.0,0.557086,150,2100,The Last Castle


# TODO
- Make global train/ global test split -- train the model appropiately and predict on test set;
- Wrap up in function recommendations - lfm_recommend();
- Calculate `NDCG@10` on test set

In [69]:
data_split =  np.random.rand(len(interactions_filtered)) < 0.8

In [70]:
data_split

array([ True, False, False, ..., False,  True, False])

In [72]:
global_train_data = interactions_filtered[data_split]
global_test_data = interactions_filtered[~data_split]

In [73]:
len(global_train_data)

35945

In [74]:
len(global_test_data)

9044

In [75]:
global_train_data

,userId,movieId,rating,timestamp
10,1,1371,2.5,1260759135
15,1,2193,2.0,1260759198
16,1,2294,2.0,1260759108
17,1,2455,2.5,1260759113
21,2,17,5.0,835355681
...,...,...,...,...
99979,671,4896,5.0,1065111996
99982,671,4993,5.0,1064245483
99983,671,4995,4.0,1064891537
99992,671,5816,4.0,1065111963


In [76]:
global_train_mat = get_coo_matrix(
    global_train_data,
    user_col = 'userId',
    item_col = 'movieId',
    users_mapping = users_mapping,
    movies_mapping = movies_mapping
    ).tocsr()

In [77]:
global_test_mat = get_coo_matrix(
    global_test_data,
    user_col = 'userId',
    item_col = 'movieId',
    users_mapping = users_mapping,
    movies_mapping = movies_mapping
    ).tocsr()

In [78]:
new_cosine_model = CosineRecommender(K = 20)
new_cosine_model.fit(global_train_mat.T)

  0%|          | 0/671 [00:00<?, ?it/s]

In [90]:
def lfm_recommend(data_matrix, N, row_id):
  recs = cosine_model.recommend(
    row_id,
    data_matrix,
    N = top_N,
    filter_already_liked_items = True
    )
  recs = pd.DataFrame(recs).T.rename(columns = {0: 'col_id', 1: 'similarity'})
  recs['inv_movie_id'] = recs['col_id'].astype(int)
  recs['movieId'] = recs['inv_movie_id'].map(movies_inv_mapping.get)
  recs['title'] = recs['movieId'].map(movie_name_mapper)

  return recs

In [103]:
  top_N = 10
  user_id1 = global_test_data['userId'].iloc[0]
  row_id1 = users_mapping[user_id]

  user_id2 = global_test_data['userId'].iloc[30]
  row_id2 = users_mapping[user_id]

In [109]:
out1 = lfm_recommend(global_test_mat, top_N, row_id1)

In [110]:
out11 = lfm_recommend(global_train_mat, top_N, row_id1)

In [111]:
out1

,col_id,similarity,inv_movie_id,movieId,title
0,337.0,0.593856,337,170,28 Days Later
1,648.0,0.577499,648,68954,Longitude
2,399.0,0.561442,399,1088,Whale Rider
3,278.0,0.561442,278,1584,School of Rock
4,150.0,0.557086,150,2100,The Last Castle
5,368.0,0.546672,368,549,Basquiat
6,384.0,0.546606,384,804,Roman Holiday
7,120.0,0.537456,120,1396,Зеркало
8,160.0,0.529829,160,2144,One Night at McCool's
9,589.0,0.525467,589,6552,Idle Hands


In [112]:
out11

,col_id,similarity,inv_movie_id,movieId,title
0,653.0,0.861587,653,74458,Mere Brother Ki Dulhan
1,129.0,0.844531,129,1994,The Most Dangerous Game
2,606.0,0.654064,606,8011,Highlander III: The Sorcerer
3,579.0,0.571681,579,5956,Joshua
4,560.0,0.550609,560,4734,Van Gogh
5,211.0,0.493337,211,41566,New York Doll
6,460.0,0.490469,460,1895,Star Wars: Episode III - Revenge of the Sith
7,517.0,0.396350,517,3083,Mr. Smith Goes to Washington
8,465.0,0.377862,465,1945,Nell
9,499.0,0.367220,499,2701,Abraham


In [113]:
from math import log2
from itertools import product

In [114]:
def compute_gain(y_value: float, gain_scheme: str) -> float:
    
    gain = {'exp2': 2 ** y_value - 1,
            'const': y_value}

    return float(gain[gain_scheme])

In [115]:
def dcg_at_k(y_true: np.array, y_pred: np.array, gain_scheme: str, k: int) -> float:

    if len(y_pred) > k:
      y_pred = y_pred[:k]
    
    dcg = 0
    argsort = np.argsort(y_pred)[::-1]
    y_true_sorted = y_true[argsort]

    for idx, val in enumerate(y_true_sorted, 1):
        gain = compute_gain(val, gain_scheme)
        dcg += gain / log2(idx + 1)
        
    return dcg

In [116]:
def ndcg_at_k(y_true: np.array, ys_pred: np.array, gain_scheme: str, k: int) -> float:
    
    # pred dcg then we calc the same to find max possible
    preds_dcg = dcg_at_k(y_true, ys_pred, gain_scheme, k)
    max_possible_dcg = dcg_at_k(y_true, y_true, gain_scheme, k)

    return preds_dcg / max_possible_dcg

In [133]:
in1 = np.array(out1['inv_movie_id'].astype(int).values.tolist())
in2 = np.array(out11['inv_movie_id'].astype(int).values.tolist())

In [134]:
ndcg_at_k(in1, in2, 'exp2', 10)

1.0